In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
I_GPU= 0

In [ ]:
# %load_ext autoreload
# %autoreload 2
import os
import os.path as osp
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'
torch.cuda.set_device(I_GPU)

In [ ]:
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
# from torch_points3d.datasets.segmentation.scannet import ScannetDataset
from torch_points3d.datasets.segmentation.multimodal.scannet import ScannetDatasetMM
from torch_points3d.datasets.segmentation.multimodal import IGNORE_LABEL

from torch_points3d.visualization.multimodal_data import visualize_mm_data

# Load config

In [ ]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/scannet'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/scannet'  # IGN DATA
# DATA_ROOT = '/media/drobert-admin/DATA2/datasets/scannet'  # IGN DATA2
# DATA_ROOT = '/var/data/drobert/datasets/scannet'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/scannet'  # CNES
DATA_ROOT = '/raid/dataset/pointcloud/data/scannet'  # ENGIE

overrides = [
    'task=segmentation',
    'data=segmentation/multimodal/scannet-sparse',
    'models=segmentation/multimodal/sparseconv3d',
    'model_name=Res16UNet34-L4-L0',
    f"data.dataroot={os.path.join(DATA_ROOT, 'testing')}",
#     f"data.dataroot={os.path.join(DATA_ROOT, 'mini')}",
    
    'data.resolution_3d=0.05',
    'data.resolution_2d=[320, 240]',
    'data.exact_splatting_2d=True'
]

cfg = hydra_read(overrides)

# print(OmegaConf.to_yaml(cfg))# Load S3DIS dataset

# Load Scannet dataset

In [ ]:
from time import time
start = time()

dataset = ScannetDatasetMM(cfg.data)
# print(dataset)

print(f"Time = {time() - start:0.1f} sec.")

In [ ]:
from torch_points3d.datasets.segmentation.scannet import VALID_CLASS_IDS, SCANNET_COLOR_MAP, CLASS_LABELS, NUM_CLASSES, IGNORE_LABEL, CLASS_COLORS, CLASS_NAMES
from torch_points3d.visualization.multimodal_data import visualize_mm_data

def remap_scannet_labels(semantic_label, valid_class_idx=VALID_CLASS_IDS, donotcare_class_ids=[]):
    """Remaps labels to [0 ; num_labels -1]."""
    new_labels = semantic_label.clone()
    mapping_dict = {idx: i for i, idx in enumerate(valid_class_idx)}
    for idx in range(NUM_CLASSES):
        if idx not in mapping_dict:
            mapping_dict[idx] = IGNORE_LABEL
    for idx in donotcare_class_ids:
        mapping_dict[idx] = IGNORE_LABEL
    for source, target in mapping_dict.items():
        mask = semantic_label == source
        new_labels[mask] = target

    broken_labels = new_labels >= len(valid_class_idx)
    new_labels[broken_labels] = IGNORE_LABEL

    return new_labels

In [ ]:
# Drop some 3D and 2D transforms to allow visualizations
dataset.train_dataset.transform.transforms = dataset.train_dataset.transform.transforms[4:-1]
dataset.train_dataset.transform_image.transforms = dataset.train_dataset.transform_image.transforms[:3]

dataset.val_dataset.transform.transforms = dataset.val_dataset.transform.transforms[:2]
dataset.val_dataset.transform_image.transforms = dataset.val_dataset.transform_image.transforms[:3]

dataset.test_dataset[0].transform.transforms = dataset.test_dataset[0].transform.transforms[:2]
dataset.test_dataset[0].transform_image.transforms = dataset.test_dataset[0].transform_image.transforms[:3]

In [ ]:
mm_data = dataset.train_dataset[0]
# mm_data = dataset.val_dataset[0]
# mm_data = dataset.test_dataset[0][0]

# Create MMData object, keep only a few images, remap labels and remove 
# IGNORE_LABEL points
# mm_data.data.y = remap_scannet_labels(mm_data.data.y)
# mm_data = mm_data[mm_data.data.y != IGNORE_LABEL]

# Prepare ScanNet colors and class names for visualization
class_colors = CLASS_COLORS
class_names = CLASS_NAMES

visualize_mm_data(mm_data, class_names=class_names, class_colors=class_colors, front='y', figsize=1000, pointsize=3, voxel=0.05, show_2d=True)

In [ ]:
import plotly

# image_idx = [20, 22, 24, 49]
image_idx = [20, 24, 49]
alpha = 0.5
palette = [hex_to_tensor(c) for c in plotly.colors.qualitative.Plotly]

# Create MMData object, keep only a few images, remap labels and remove 
# IGNORE_LABEL points
mm_data_less = MMData(data_out.clone(), image=ImageData([image_data_out.clone()[image_idx]]))
mm_data_less.data.y = remap_scannet_labels(mm_data_less.data.y)
mm_data_less = mm_data_less[mm_data_less.data.y != IGNORE_LABEL]

# Color the points wrt the images that see them, following the plotly
# palette used to color image balls
for i in range(len(image_idx)):
    mapping = mm_data_less.modalities['image'][0].mappings.select_images(i)
    idx_seen = mapping.pointers[1:] > mapping.pointers[:-1]
    mm_data_less.data.rgb[idx_seen] = mm_data_less.data.rgb[idx_seen] * (1 - alpha) + palette[i % len(palette)] * alpha

# Prepare ScanNet colors and class names for visualization
class_colors = list(SCANNET_COLOR_MAP.values())
class_names = CLASS_LABELS

visualize_mm_data(mm_data_less, class_names=class_names, class_colors=class_colors, figsize=1000, pointsize=3, voxel=0.05)

# Fiddling with poses

In [ ]:
import plotly
import plotly.graph_objects as go

def rgb_to_plotly_rgb(rgb):
    """Convert torch.Tensor of float RGB values in [0, 1] to
    plotly-friendly RGB format.
    """
    assert isinstance(rgb, torch.Tensor) and rgb.max() <= 1.0 and rgb.dim() <= 2

    if rgb.dim() == 1:
        rgb = rgb.unsqueeze(0)

    return [f"rgb{tuple(x)}" for x in (rgb * 255).int().numpy()]

def hex_to_tensor(h):
    h = h.lstrip('#')
    rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
    return torch.Tensor(rgb) / 255

def plot3d(data, image_xyz=None):
    # Prepare figure
    width = 600
    height = 400
    margin = int(0.02 * min(width, height))
    pointsize = 3
    layout = go.Layout(
        width=width,
        height=height,
        scene=dict(aspectmode='data', ),  # preserve aspect ratio
        margin=dict(l=margin, r=margin, b=margin, t=margin),
        uirevision=True)
    fig = go.Figure(layout=layout)
    initialized_visibility = False

    # Draw a trace for RGB 3D point cloud
    if getattr(data, 'rgb', None) is not None:
        fig.add_trace(
            go.Scatter3d(
                name='RGB',
                x=data.pos[:, 0],
                y=data.pos[:, 1],
                z=data.pos[:, 2],
                mode='markers',
                marker=dict(
                    size=pointsize,
                    color=rgb_to_plotly_rgb(data.rgb), ),
                hoverinfo='x+y+z',
                showlegend=False,
                visible=True, ))
        
    if image_xyz is None:
        fig.show(config={'displayModeBar': False})
        return
    
    if len(image_xyz.shape) == 1:
        image_xyz = image_xyz.reshape((1, -1))
    for i, xyz in enumerate(image_xyz):
        # Draw image position as ball
        fig.add_trace(
            go.Scatter3d(
                name=f"Image {i}",
                x=[xyz[0]],
                y=[xyz[1]],
                z=[xyz[2]],
                mode='markers+text',
                marker=dict(
                    line_width=2,
                    size=pointsize + 4, 
                    color=plotly.colors.qualitative.Plotly[i], ),
                text=f"<b>{i}</b>",
                textposition="bottom center",
                textfont=dict(
                    size=16),
                hoverinfo='x+y+z+name',
                showlegend=False,
                visible=True, ))
        

    fig.show(config={'displayModeBar': False})

In [ ]:
all_data, all_slices = torch.load('/media/drobert-admin/DATA2/datasets/scannet/testing/scannet-sparse/processed/preprocessed_3d_train.pt')

In [ ]:
idx = 0
data = all_data.__class__(
    pos=all_data.pos[all_slices['pos'][idx]: all_slices['pos'][idx+1]], 
    rgb=all_data.rgb[all_slices['rgb'][idx]: all_slices['rgb'][idx+1]], 
    y=all_data.y[all_slices['y'][idx]: all_slices['y'][idx+1]])

from torch_points3d.core.data_transform.grid_transform import GridSampling3D
data = GridSampling3D(0.1)(data)

In [ ]:
plot3d(data)

In [ ]:
from torch_points3d.datasets.segmentation.multimodal.scannet import load_pose, read_image_pose_pairs
import os
import os.path as osp
import glob
from PIL import Image
from matplotlib import pyplot as plt

scannet_dir = '/media/drobert-admin/DATA2/datasets/scannet/testing/scannet-sparse/raw/scans'
scan_name = 'scene0000_00'
scan_sens_dir = osp.join(scannet_dir, scan_name, 'sens')

image_info_list = [
    {'path': i_file, 'extrinsic': load_pose(p_file)}
    for i_file, p_file in read_image_pose_pairs(
        osp.join(scan_sens_dir, 'color'),
        osp.join(scan_sens_dir, 'pose'),
        image_suffix='.jpg', pose_suffix='.txt', verbose=True)]

intrinsic = load_pose(osp.join(scan_sens_dir, 'intrinsic/intrinsic_color.txt'))

In [ ]:
import math

# create camera intrinsics
def make_intrinsic(fx, fy, mx, my):
    intrinsic = np.eye(4)
    intrinsic[0][0] = fx
    intrinsic[1][1] = fy
    intrinsic[0][2] = mx
    intrinsic[1][2] = my
    return intrinsic


# create camera intrinsics
def adjust_intrinsic(intrinsic, intrinsic_image_dim, image_dim):
    if intrinsic_image_dim == image_dim:
        return intrinsic
    resize_width = int(math.floor(image_dim[1] * float(intrinsic_image_dim[0]) / float(intrinsic_image_dim[1])))
    intrinsic[0, 0] *= float(resize_width) / float(intrinsic_image_dim[0])
    intrinsic[1, 1] *= float(image_dim[1]) / float(intrinsic_image_dim[1])
    # account for cropping here
    intrinsic[0, 2] *= float(image_dim[0] - 1) / float(intrinsic_image_dim[0] - 1)
    intrinsic[1, 2] *= float(image_dim[1] - 1) / float(intrinsic_image_dim[1] - 1)
    return intrinsic


class LinkCreator(object):
    def __init__(self, intrinsic, image_dim=(320, 240), voxelSize=0.05):
#         self.intrinsic = make_intrinsic(fx=fx, fy=fy, mx=mx, my=my)
        self.intrinsic = intrinsic
        self.intrinsic = adjust_intrinsic(self.intrinsic, intrinsic_image_dim=[640, 480], image_dim=image_dim)
        self.imageDim = image_dim
        self.voxel_size = voxelSize

    def computeLinking(self, camera_to_world, coords):
        """
        :param camera_to_world: 4 x 4
        :param coords: N x 3 format
        :param depth: H x W format
        :return: linking, N x 3 format, (H,W,mask)
        """
#         link = torch.zeros((3, coords.shape[0]), dtype=np.int)
        out = torch.zeros((3, coords.shape[0]), dtype=torch.long)
        coordsNew = torch.cat([coords, torch.ones([coords.shape[0], 1])], dim=1).T
        assert coordsNew.shape[0] == 4, "[!] Shape error"

        world_to_camera = torch.inverse(camera_to_world)
        p = world_to_camera.mm(coordsNew)
        p[0] = (p[0] * self.intrinsic[0][0]) / p[2] + self.intrinsic[0][2]
        p[1] = (p[1] * self.intrinsic[1][1]) / p[2] + self.intrinsic[1][2]
        pi = torch.round(p).long()
        inside_mask = (pi[0] >= 0) * (pi[1] >= 0) * (pi[0] < self.imageDim[0]) * (pi[1] < self.imageDim[1]) * (p[2] > 0)
        
        # TODO : which one is it ?
        out[0][inside_mask] = pi[0][inside_mask]
        out[1][inside_mask] = pi[1][inside_mask]
        out[2][inside_mask] = 1
        
        return out.T

In [ ]:
# Trying to find image centers
image_xyz = []
for i in range(len(image_info_list)):
    
    P = image_info_list[i]['extrinsic']
    R = P[:3, :3]
    T = P[:3, 3]
    C = -R.T.mm(T.view(-1, 1))
    image_xyz.append(C.view(1, -1))
image_xyz = torch.cat(image_xyz, dim=0)

plot3d(data, image_xyz=image_xyz)

In [ ]:
# Trying to find image centers
image_xyz = torch.cat([image_info_list[i]['extrinsic'][:3, [3]].T for i in range(len(image_info_list))], dim=0)

plot3d(data, image_xyz=image_xyz)

In [ ]:
# Trying to find image centers
axis_align_matrix = torch.Tensor([0.945519, 0.325568, 0.000000, -5.384390, -0.325568, 0.945519, 0.000000, -2.871780, 0.000000, 0.000000, 1.000000, -0.064350, 0.000000, 0.000000, 0.000000, 1.000000]).reshape((4, 4))

xyzh = torch.cat([image_info_list[i]['extrinsic'][:, [3]].T for i in range(len(image_info_list))], dim=0)
# xyzh = torch.ones_like(image_xyz)
# xyzh[:, :3] = image_xyz
xyzh = xyzh.mm(axis_align_matrix.T)
image_xyz = xyzh[:, :3]

plot3d(data, image_xyz=image_xyz)

In [ ]:
idx = 0

LC = LinkCreator(load_pose(osp.join(scan_sens_dir, 'intrinsic/intrinsic_color.txt')), image_dim=(1296, 968), voxelSize=0.05)
link = LC.computeLinking(image_info_list[idx]['extrinsic'], data.pos)

data_v = data.clone()
data_v.rgb[link[:, 2] == 1] = data_v.rgb[link[:, 2] == 1] * 0.5 + torch.Tensor([0.5, 0, 0])

plot3d(data_v, image_xyz=image_xyz[0])

In [ ]:
idx = 0
data = all_data.__class__(
    pos=all_data.pos[all_slices['pos'][idx]: all_slices['pos'][idx+1]], 
    rgb=all_data.rgb[all_slices['rgb'][idx]: all_slices['rgb'][idx+1]], 
    y=all_data.y[all_slices['y'][idx]: all_slices['y'][idx+1]])

In [ ]:
axis_align_matrix = torch.Tensor([0.945519, 0.325568, 0.000000, -5.384390, -0.325568, 0.945519, 0.000000, -2.871780, 0.000000, 0.000000, 1.000000, -0.064350, 0.000000, 0.000000, 0.000000, 1.000000]).reshape((4, 4))

indices = [0, 5, 10, 20, 30]
palette = [hex_to_tensor(c) for c in plotly.colors.qualitative.Plotly]
rgb_size = (1296, 968)
proj_size = (320, 240)

intrinsic = load_pose(osp.join(scan_sens_dir, 'intrinsic/intrinsic_depth.txt'))
LC = LinkCreator(intrinsic, image_dim=proj_size)

image_xyz_list = []
image_target_list = []
image_proj_list = []
data_v = data.clone()
alpha = 0.5

for i, idx in enumerate(indices):
    
    # Compute projection
    extrinsic = axis_align_matrix.mm(image_info_list[idx]['extrinsic'])
    link = LC.computeLinking(extrinsic, data.pos)
    
    # Compute
    color = palette[i % len(palette)]
    data_v.rgb[link[:, 2] == 1] = (data_v.rgb[link[:, 2] == 1]) * (1 - alpha) + color * alpha
    
    image_xyz_list.append(extrinsic[:3, 3].view(1, -1))

    # Recover target RGB
    image_target = np.asarray(Image.open(image_info_list[idx]['path']).resize(proj_size))
    image_target_list.append(image_target)

    # Compute artificial RGB image from projected points
    mask = link[:, 2] == 1
    image_proj = torch.zeros((*proj_size, 3)).byte()
    image_proj[link[mask, 0], link[mask, 1]] = (data.rgb[mask] * 255).byte()
    image_proj_list.append(image_proj)

plot3d(data_v, image_xyz=torch.cat(image_xyz_list, dim=0))

# Draw images
for i, (image_target, image_proj) in enumerate(zip(image_target_list, image_proj_list)):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,3.5))
    fig.suptitle(f"Image {i}")
    ax1.imshow(image_target)
    ax2.imshow(image_proj.transpose(0, 1))
    plt.show()

# Handle intrinsics and extrinsics in SameSettingImageData

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
I_GPU = 0

In [ ]:
# %load_ext autoreload
# %autoreload 2
import os
import os.path as osp
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'
torch.cuda.set_device(I_GPU)

In [ ]:
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
# from torch_points3d.datasets.segmentation.scannet import ScannetDataset
from torch_points3d.datasets.segmentation.multimodal.scannet import ScannetDatasetMM
from torch_points3d.datasets.segmentation.multimodal import IGNORE_LABEL

from torch_points3d.visualization.multimodal_data import visualize_mm_data, hex_to_tensor

In [ ]:
from torch_points3d.datasets.segmentation.multimodal.scannet import load_pose, read_image_pose_pairs
from torch_points3d.core.data_transform.grid_transform import GridSampling3D
import os
import os.path as osp
import glob
from PIL import Image
from matplotlib import pyplot as plt

scannet_dir = '/media/drobert-admin/DATA2/datasets/scannet/testing/scannet-sparse/raw/scans'
scan_name = 'scene0000_00'
scan_sens_dir = osp.join(scannet_dir, scan_name, 'sens')

all_data, all_slices = torch.load('/media/drobert-admin/DATA2/datasets/scannet/testing/scannet-sparse/processed/preprocessed_3d_train.pt')

idx = 0
data = all_data.__class__(
    pos=all_data.pos[all_slices['pos'][idx]: all_slices['pos'][idx+1]], 
    rgb=all_data.rgb[all_slices['rgb'][idx]: all_slices['rgb'][idx+1]], 
    y=all_data.y[all_slices['y'][idx]: all_slices['y'][idx+1]])


n_full = data.num_nodes
data = GridSampling3D(0.05)(data)
n_sampled = data.num_nodes
print(f'{n_sampled / n_full * 100:0.1f}% subsampled')

In [ ]:
# from torch_points3d.datasets.segmentation.scannet import read_axis_align_matrix

def read_axis_align_matrix(filename):
    lines = open(filename).readlines()
    axis_align_matrix = None
    for line in lines:
        if "axisAlignment" in line:
            axis_align_matrix = torch.Tensor([float(x) for x in line.rstrip().strip("axisAlignment = ").split(" ")]).reshape((4, 4))
            break
    return axis_align_matrix

In [ ]:
scan_sens_dir = osp.join(scannet_dir, scan_name, 'sens')
meta_file = osp.join(scannet_dir, scan_name, scan_name + ".txt")
    
image_info_list = [
    {'path': i_file, 'extrinsic': load_pose(p_file)}
    for i_file, p_file in read_image_pose_pairs(
        osp.join(scan_sens_dir, 'color'),
        osp.join(scan_sens_dir, 'pose'),
        image_suffix='.jpg', pose_suffix='.txt')]

# Aggregate all RGB image paths
path = np.array([info['path'] for info in image_info_list])

# Aggregate all extrinsic 4x4 matrices
# Train and val scans have undergone axis alignment
# transformations. Need to recover and apply those
# to camera poses too. Test scans have no axis
# alignment
axis_align_matrix = read_axis_align_matrix(meta_file)
if axis_align_matrix is not None:
    extrinsic = torch.cat([
        axis_align_matrix.mm(info['extrinsic']).unsqueeze(0)
        for info in image_info_list], dim=0)
else:
    extrinsic = torch.cat([
        info['extrinsic'].unsqueeze(0)
        for info in image_info_list], dim=0)

# For easier image handling, extract the images
# position from the extrinsic matrices
xyz = extrinsic[:, :3, 3]

# Read intrinsic parameters for the depth camera
# because this is the one related to the extrinsic.
# Strangely, using the color camera intrinsic along
# with the pose does not produce the expected
# projection
intrinsic = load_pose(osp.join(scan_sens_dir, 'intrinsic/intrinsic_depth.txt'))
fx = intrinsic[0][0].repeat(len(image_info_list))
fy = intrinsic[1][1].repeat(len(image_info_list))
mx = intrinsic[0][2].repeat(len(image_info_list))
my = intrinsic[1][2].repeat(len(image_info_list))

In [ ]:
from torch_points3d.core.multimodal.image import SameSettingImageData

DEPTH_IMG_SIZE = (640, 480)
img_size = (320, 240)
image_data = SameSettingImageData(ref_size=DEPTH_IMG_SIZE, path=path, pos=xyz, fx=fx, fy=fy, mx=mx, my=my, proj_upscale=1, extrinsic=extrinsic)
image_data.ref_size = img_size
image_data

In [ ]:
from torch_points3d.core.data_transform.multimodal.image import MapImages
from torch_points3d.core.multimodal.data import MMData
from torch_points3d.core.multimodal.image import ImageData

transform = MapImages(method='SplattingVisibility', r_max=8, r_min=0.5, equirectangular=False, voxel=0.05, k_swell=1.0, d_swell=1000, exact=True)

data.mapping_index = torch.arange(data.num_nodes).long()
data_out, image_data_out = transform(data, image_data)

mm_data = MMData(data_out, image=ImageData([image_data_out]))
mm_data

In [ ]:
visualize_mm_data(mm_data, figsize=1000, pointsize=3, voxel=0.05)

In [ ]:
from torch_points3d.datasets.segmentation.scannet import VALID_CLASS_IDS, SCANNET_COLOR_MAP, CLASS_LABELS, NUM_CLASSES, IGNORE_LABEL

def remap_scannet_labels(semantic_label, valid_class_idx=VALID_CLASS_IDS, donotcare_class_ids=[]):
    """Remaps labels to [0 ; num_labels -1]."""
    new_labels = semantic_label.clone()
    mapping_dict = {idx: i for i, idx in enumerate(valid_class_idx)}
    for idx in range(NUM_CLASSES):
        if idx not in mapping_dict:
            mapping_dict[idx] = IGNORE_LABEL
    for idx in donotcare_class_ids:
        mapping_dict[idx] = IGNORE_LABEL
    for source, target in mapping_dict.items():
        mask = semantic_label == source
        new_labels[mask] = target

    broken_labels = new_labels >= len(valid_class_idx)
    new_labels[broken_labels] = IGNORE_LABEL

    return new_labels

In [ ]:
# Create MMData object, keep only a few images, remap labels and remove 
# IGNORE_LABEL points
mm_data_less = MMData(data_out.clone(), image=ImageData([image_data_out.clone()[:5]]))
mm_data_less.data.y = remap_scannet_labels(mm_data_less.data.y)
mm_data_less = mm_data_less[mm_data_less.data.y != IGNORE_LABEL]

# Prepare ScanNet colors and class names for visualization
class_colors = list(SCANNET_COLOR_MAP.values())
class_names = CLASS_LABELS

visualize_mm_data(mm_data_less, class_names=class_names, class_colors=class_colors, front='y', figsize=1000, pointsize=3, voxel=0.05)

In [ ]:
import plotly

# image_idx = [20, 22, 24, 49]
image_idx = [20, 24, 49]
alpha = 0.5
palette = [hex_to_tensor(c) for c in plotly.colors.qualitative.Plotly]

# Create MMData object, keep only a few images, remap labels and remove 
# IGNORE_LABEL points
mm_data_less = MMData(data_out.clone(), image=ImageData([image_data_out.clone()[image_idx]]))
mm_data_less.data.y = remap_scannet_labels(mm_data_less.data.y)
mm_data_less = mm_data_less[mm_data_less.data.y != IGNORE_LABEL]

# Color the points wrt the images that see them, following the plotly
# palette used to color image balls
for i in range(len(image_idx)):
    mapping = mm_data_less.modalities['image'][0].mappings.select_images(i)
    idx_seen = mapping.pointers[1:] > mapping.pointers[:-1]
    mm_data_less.data.rgb[idx_seen] = mm_data_less.data.rgb[idx_seen] * (1 - alpha) + palette[i % len(palette)] * alpha

# Prepare ScanNet colors and class names for visualization
class_colors = list(SCANNET_COLOR_MAP.values())
class_names = CLASS_LABELS

visualize_mm_data(mm_data_less, class_names=class_names, class_colors=class_colors, figsize=1000, pointsize=3, voxel=0.05)

# Simulate preprocessing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
I_GPU = 0

In [ ]:
# %load_ext autoreload
# %autoreload 2
import os
import os.path as osp
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'
torch.cuda.set_device(I_GPU)

In [ ]:
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
# from torch_points3d.datasets.segmentation.scannet import ScannetDataset
from torch_points3d.datasets.segmentation.multimodal.scannet import ScannetDatasetMM
from torch_points3d.datasets.segmentation.multimodal import IGNORE_LABEL

from torch_points3d.visualization.multimodal_data import visualize_mm_data, hex_to_tensor

In [ ]:
from torch_points3d.datasets.segmentation.multimodal.scannet import load_pose, read_image_pose_pairs
from torch_points3d.core.data_transform.grid_transform import GridSampling3D
import os
import os.path as osp
import glob
from PIL import Image
from matplotlib import pyplot as plt

scannet_dir = '/media/drobert-admin/DATA2/datasets/scannet/testing/scannet-sparse/raw/scans'
scan_name = 'scene0000_00'
scan_sens_dir = osp.join(scannet_dir, scan_name, 'sens')

all_data, all_slices = torch.load('/media/drobert-admin/DATA2/datasets/scannet/testing/scannet-sparse/processed/preprocessed_3d_train.pt')

idx = 0
data = all_data.__class__(
    pos=all_data.pos[all_slices['pos'][idx]: all_slices['pos'][idx+1]], 
    rgb=all_data.rgb[all_slices['rgb'][idx]: all_slices['rgb'][idx+1]], 
    y=all_data.y[all_slices['y'][idx]: all_slices['y'][idx+1]])

n_full = data.num_nodes
data = GridSampling3D(0.05)(data)
n_sampled = data.num_nodes
print(f'{n_sampled / n_full * 100:0.1f}% subsampled')

In [ ]:
# from torch_points3d.datasets.segmentation.scannet import read_axis_align_matrix

def read_axis_align_matrix(filename):
    lines = open(filename).readlines()
    axis_align_matrix = None
    for line in lines:
        if "axisAlignment" in line:
            axis_align_matrix = torch.Tensor([float(x) for x in line.rstrip().strip("axisAlignment = ").split(" ")]).reshape((4, 4))
            break
    return axis_align_matrix

In [ ]:
scan_sens_dir = osp.join(scannet_dir, scan_name, 'sens')
meta_file = osp.join(scannet_dir, scan_name, scan_name + ".txt")
    
image_info_list = [
    {'path': i_file, 'extrinsic': load_pose(p_file)}
    for i_file, p_file in read_image_pose_pairs(
        osp.join(scan_sens_dir, 'color'),
        osp.join(scan_sens_dir, 'pose'),
        image_suffix='.jpg', pose_suffix='.txt')]

# Aggregate all RGB image paths
path = np.array([info['path'] for info in image_info_list])

# Aggregate all extrinsic 4x4 matrices
# Train and val scans have undergone axis alignment
# transformations. Need to recover and apply those
# to camera poses too. Test scans have no axis
# alignment
axis_align_matrix = read_axis_align_matrix(meta_file)
if axis_align_matrix is not None:
    extrinsic = torch.cat([
        axis_align_matrix.mm(info['extrinsic']).unsqueeze(0)
        for info in image_info_list], dim=0)
else:
    extrinsic = torch.cat([
        info['extrinsic'].unsqueeze(0)
        for info in image_info_list], dim=0)

# For easier image handling, extract the images
# position from the extrinsic matrices
xyz = extrinsic[:, :3, 3]

# Read intrinsic parameters for the depth camera
# because this is the one related to the extrinsic.
# Strangely, using the color camera intrinsic along
# with the pose does not produce the expected
# projection
intrinsic = load_pose(osp.join(scan_sens_dir, 'intrinsic/intrinsic_depth.txt'))
fx = intrinsic[0][0].repeat(len(image_info_list))
fy = intrinsic[1][1].repeat(len(image_info_list))
mx = intrinsic[0][2].repeat(len(image_info_list))
my = intrinsic[1][2].repeat(len(image_info_list))

In [ ]:
from torch_points3d.core.multimodal.image import SameSettingImageData

DEPTH_IMG_SIZE = (640, 480)
img_size = (320, 240)
image_data = SameSettingImageData(ref_size=DEPTH_IMG_SIZE, path=path, pos=xyz, fx=fx, fy=fy, mx=mx, my=my, proj_upscale=1, extrinsic=extrinsic)
image_data.ref_size = img_size
image_data

In [ ]:
from torch_points3d.core.data_transform.multimodal.image import MapImages
from torch_points3d.core.multimodal.data import MMData
from torch_points3d.core.multimodal.image import ImageData

transform = MapImages(method='SplattingVisibility', r_max=8, r_min=0.5, equirectangular=False, voxel=0.05, k_swell=1.0, d_swell=1000, exact=True)

data.mapping_index = torch.arange(data.num_nodes).long()
data_out, image_data_out = transform(data, image_data)

mm_data = MMData(data_out, image=ImageData([image_data_out]))
mm_data